In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.4/533.4 kB 3.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 5.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.0/36.0 MB 7.9 MB/s  0:00:04m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31/31 [datasets]/31 [datasets]ce-hub]


In [ ]:
# Load the TinyStories dataset
from datasets import load_dataset
ds = load_dataset("roneneldan/TinyStories")

Training data = ~2M,
Test split = 21K (1%)

Step 2: Tokenization

Tokenize the dataset into token IDs
Create train.bin / test.bin files where the token IDs for the whole dataset is stored
Need to store on disk using memory-mapped array rather than RAM (Probably bigger machines can store on Local SSD)

In [ ]:
import tiktoken # different tokenizers
import os
import numpy as np
from tqdm.auto import tqdm

enc = tiktoken.get_encoding("gpt2") # byte pair encoding

def process(example):
  ids = enc.encode_ordinary(example['text'])
  out = {'ids': ids, 'len': len(ids)}
  return out

# testing to see if tokens generated
print(process({'text': 'The quick brown fox jumps over the lazy dog.'}))

In [ ]:
if not os.path.exists("train.bin"):
  tokenized = ds.map(process, remove_columns=['text'], desc="tokenization process", num_proc=8)

print(tokenized)

Runtime of above process on T4 GPU = ~5min
Creates a Dataset object like so:

DatasetDict({
    train: Dataset({
        features: ['ids', 'len'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['ids', 'len'],
        num_rows: 21990
    })
})

In [ ]:
for split, dset in tokenized.items():
  arr_len = np.sum(dset['len'], dtype=np.uint64)
  filename = f'{split}.bin'
  dtype = np.uint16 # enc.max_token_value in GPT2 (Byte Pair Encoding) < 2^16, so all can fit
  arr = np.memmap(filename, dtype, mode='w+', shape=(arr_len,))
  total_batches = 1024 # based on GPT2 paper, can be modified

  # For each split, we combine the ids in batches, and concatenate all batched arrays
  idx = 0
  for batch_idx in tqdm(range(total_batches)):
    batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True)
    arr_batch = np.concatenate(batch['ids'])
    arr[idx: idx + len(arr_batch)] = arr_batch
    idx += len(arr_batch)
  
  arr.flush()

print(arr)

Runtime of batch processing = ~15min